### Define funtions for SQL queries

In [3]:
import sqlite3
from sqlite3 import Error
import pandas as pd

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        #print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def perform_sql(query1):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    
    cur = conn.cursor()
    
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    return(pd.DataFrame(rows))




### Obtaining information for each table <font color='red'>(HOW TO RETURN RESULTS WITH COLNAMES IN A QUERY?)</font>

In [11]:
'''
BOOKINGS Table
'''

query1 = 'PRAGMA table_info(BOOKINGS)'

perform_sql(query1)

,0,1,2,3,4,5
0,0,bookid,int(4),1,'0',1
1,1,facid,int(1),0,NULL,0
2,2,memid,int(2),0,NULL,0
3,3,starttime,varchar(19),0,NULL,0
4,4,slots,int(2),0,NULL,0


In [12]:
'''
FACILTIES Table
'''

query1 = 'PRAGMA table_info(FACILITIES)'

perform_sql(query1)

,0,1,2,3,4,5
0,0,facid,int(1),1,'0',1
1,1,name,varchar(15),0,NULL,0
2,2,membercost,"decimal(2,1)",0,NULL,0
3,3,guestcost,"decimal(3,1)",0,NULL,0
4,4,initialoutlay,int(5),0,NULL,0
5,5,monthlymaintenance,int(4),0,NULL,0


In [13]:
'''
MEMBERS Table
'''

query1 = 'PRAGMA table_info(MEMBERS)'

perform_sql(query1)

,0,1,2,3,4,5
0,0,memid,int(2),1,'0',1
1,1,surname,varchar(17),0,NULL,0
2,2,firstname,varchar(9),0,NULL,0
3,3,address,varchar(39),0,NULL,0
4,4,zipcode,int(5),0,NULL,0
5,5,telephone,varchar(14),0,NULL,0
6,6,recommendedby,varchar(2),0,NULL,0
7,7,joindate,varchar(19),0,NULL,0


In [14]:
'''/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */'''


queryQ1 = 'SELECT name FROM FACILITIES WHERE membercost>0;'

perform_sql(queryQ1)

,0
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


In [15]:
'''
/* Q2: How many facilities do not charge a fee to members? */
'''
queryQ2 = 'SELECT COUNT(*) FROM FACILITIES WHERE membercost>0;'

perform_sql(queryQ2)


,0
0,5


In [18]:
'''/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */'''

queryQ3 = 'SELECT facid, name, membercost, monthlymaintenance FROM FACILITIES \
WHERE membercost < 0.2*monthlymaintenance AND membercost > 0'

perform_sql(queryQ3)

,0,1,2,3
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


In [21]:
'''
/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */
'''

queryQ4 = 'SELECT * FROM FACILITIES WHERE facid IN (1,5)'

perform_sql(queryQ4)

,0,1,2,3,4,5
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


In [23]:
'''
/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */
'''
queryQ5 = "SELECT name, monthlymaintenance, \
(CASE WHEN monthlymaintenance>100 THEN 'expensive' ELSE 'cheap' END) AS costRank \
FROM FACILITIES"

perform_sql(queryQ5)

,0,1,2
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


In [27]:
'''
/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */
'''
queryQ6 = 'SELECT firstname, surname MEMBERS FROM MEMBERS WHERE joindate=(SELECT MAX(joindate) FROM MEMBERS)'

perform_sql(queryQ6)

,0,1
0,Darren,Smith


In [40]:
'''
/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */
'''
queryQ7 = "SELECT m.firstname || ' ' || m.surname AS member, f.name AS facility FROM MEMBERS AS m \
INNER JOIN (SELECT * FROM BOOKINGS) USING (memid) INNER JOIN \
(SELECT * FROM FACILITIES WHERE name LIKE 'Tennis%') AS f USING (facid) \
ORDER BY member DESC"

perform_sql(queryQ7)


,0,1
0,Tracy Smith,Tennis Court 1
1,Tracy Smith,Tennis Court 1
2,Tracy Smith,Tennis Court 1
3,Tracy Smith,Tennis Court 1
4,Tracy Smith,Tennis Court 1
...,...,...
792,Anne Baker,Tennis Court 1
793,Anne Baker,Tennis Court 2
794,Anne Baker,Tennis Court 2
795,Anne Baker,Tennis Court 2


In [58]:
'''
/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */
'''

queryQ8 = "SELECT f.name AS facility, m.firstname || ' ' || m.surname AS member, \
(CASE WHEN b.memid = 0 THEN b.slots*f.guestcost ELSE b.slots*f.membercost END) AS cost \
FROM BOOKINGS AS b INNER JOIN MEMBERS AS m USING(memid) \
INNER JOIN FACILITIES AS f ON f.facid = b.facid WHERE cost > 30 AND date(b.starttime) = '2012-09-14' ORDER BY cost DESC"

perform_sql(queryQ8)

,0,1,2
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


In [59]:
'''
/* Q9: This time, produce the same result as in Q8, but using a subquery. */

'''
queryQ9 = "SELECT name AS facility, firstname || ' ' || surname AS member, \
(CASE WHEN memid = 0 THEN slots*guestcost ELSE slots*membercost END) AS cost \
FROM (SELECT * FROM BOOKINGS AS b INNER JOIN MEMBERS AS m USING(memid) \
INNER JOIN FACILITIES AS f ON f.facid = b.facid) WHERE cost > 30 AND date(starttime) = '2012-09-14' \
ORDER BY cost DESC"

perform_sql(queryQ9)



,0,1,2
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


### Easier solution for Q10????

In [102]:
'''
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */
'''

queryQ10 = "WITH t AS (SELECT name, memid, (CASE WHEN memid=0 THEN guestcost*slots ELSE \
membercost*slots END) AS income, slots, monthlymaintenance, f.facid, \
strftime('%m', starttime) AS month FROM BOOKINGS AS b INNER JOIN \
MEMBERS AS m USING (memid) INNER JOIN FACILITIES AS f on f.facid = b.facid) \
SELECT DISTINCT name, total_revenue FROM (SELECT name, SUM(total_income-monthlymaintenance) OVER(PARTITION BY name) \
AS total_revenue FROM (SELECT DISTINCT name, SUM(income) OVER(PARTITION BY name, month) \
AS total_income, month, monthlymaintenance FROM t)) WHERE total_revenue<1000 \
ORDER BY total_revenue"
perform_sql(queryQ10)

,0,1
0,Table Tennis,150
1,Snooker Table,195
2,Pool Table,225


### Best solution for Q11???

In [125]:
'''
/* Q11: Produce a report of members and who recommended them in alphabetic 
surname,firstname order */
'''
queryQ11 = "WITH joinedT AS (SELECT * FROM MEMBERS WHERE memid>0 AND recommendedby >0)\
SELECT joinedT.surname || ', ' || joinedT.firstname AS member, \
 m.surname || ', ' || m.firstname AS recomender FROM joinedT \
INNER JOIN MEMBERS AS m on joinedT.recommendedby = m.memid \
ORDER BY member"

perform_sql(queryQ11)


,0,1
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


In [133]:
'''
/* Q12: Find the facilities with their usage by member, but not guests */
'''

queryQ12 = 'SELECT DISTINCT f.name, SUM(slots) OVER(PARTITION BY facid) AS usage \
FROM FACILITIES AS f INNER JOIN (SELECT * FROM BOOKINGS \
WHERE memid>0) AS m USING (facid) ORDER BY usage DESC'

perform_sql(queryQ12)

,0,1
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


In [138]:
'''
/* Q13: Find the facilities usage by month, but not guests */
'''
queryQ13 = "SELECT DISTINCT month, SUM(slots) OVER(PARTITION BY month) AS usage \
FROM (SELECT strftime('%m', starttime) AS month, slots FROM FACILITIES AS f INNER JOIN \
(SELECT * FROM BOOKINGS \
WHERE memid>0) AS m USING (facid)) ORDER BY usage DESC"

perform_sql(queryQ13)

,0,1
0,09,3199
1,08,2531
2,07,1061
